In [1]:
import os

In [2]:
%pwd


'D:\\ML projects\\ML class\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")
%pwd

'D:\\ML projects\\ML class\\Kidney-Disease-Classification'

In [32]:
import dagshub
dagshub.init(repo_owner='mcmlxiv', repo_name='Kidney-Disease-Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "mcmlxiv/Kidney-Disease-Classification"

[2024-05-27 18:02:32,051: INFO: helpers]: Initialized MLflow to track repo "mcmlxiv/Kidney-Disease-Classification"


Repository mcmlxiv/Kidney-Disease-Classification initialized!

[2024-05-27 18:02:32,062: INFO: helpers]: Repository mcmlxiv/Kidney-Disease-Classification initialized!


In [33]:
import tensorflow as tf

In [33]:
model =tf.keras.models.load_model('artifacts/training/model.h5')

In [34]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int
  

In [35]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories, save_json

In [36]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath= Path("D:\ML projects\ML class\Kidney-Disease-Classification\params.yaml"),
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self):
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion",
            mlflow_uri = "https://dagshub.com/mcmlxiv/Kidney-Disease-Classification.mlflow",
            all_params = self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
        )
        return eval_config

In [37]:
%pwd

'D:\\ML projects\\ML class\\Kidney-Disease-Classification'

In [38]:
import os
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [39]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [40]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

[2024-05-27 18:02:39,215: INFO: common]: yaml file: config\config.yaml loaded successfully
[2024-05-27 18:02:39,220: INFO: common]: yaml file: D:\ML projects\ML class\Kidney-Disease-Classification\params.yaml loaded successfully
[2024-05-27 18:02:39,222: INFO: common]: created directory at: artifacts
Found 138 images belonging to 2 classes.
9/9 [==============================] - 23s 2s/step - loss: 39.2283 - accuracy: 0.4783
[2024-05-27 18:03:02,965: INFO: common]: json files saved at: scores.json


AttributeError: 'EvaluationConfig' object has no attribute 'mlflow_uri'